In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import yfinance as yf
import datetime
from IPython.display import clear_output
import pytz
import timeit
from time import sleep

#import plotly.io as pio
#pio.renderers.default = "browser"

In [2]:
def resample_data(df, interval):
    # Define the resampling logic
    ohlcv_dict = {
         'Open': 'first',
         'High': 'max',
         'Low': 'min',
         'Close': 'last',
         'Volume': 'sum'
    }
    # 10T for 10 minutes, D for 1 day and M for 1 month
    # Resample the data
    df = df.resample(interval).agg(ohlcv_dict)
    df = df.dropna()
    return df

In [3]:
import pandas_ta as ta
import yfinance.shared as shared

def create_watchlist_dict(watchlist_path):
    # Retrieve ticker's symbol from watchlist
    watchlist_filename = watchlist_path
    df_watchlist = pd.read_csv(watchlist_filename)  
    print(df_watchlist)

    # Create a dictionary 
    watchlist_dict = pd.Series(None,df_watchlist.Symbol.values).to_dict()
    # Initialize empty dataframe
    for key, df in watchlist_dict.items():
        watchlist_dict[key] = pd.DataFrame()
        
    return watchlist_dict

# Calculate indicators EMA9, EMA13, VWAP
def vwap_indicator(df):
    v = df.Volume.values
    tp = (df.High+df.Low+df.Close).div(3).values
    df['vwap'] = (tp*v).cumsum() / v.cumsum()
    return df

def calculate_indicators(df):
    df = df.groupby(df.index.date, group_keys=False).apply(vwap_indicator)
    df['ema_nine'] = ta.ema(df['Close'], length=9)
    df['ema_thirteen'] = ta.ema(df['Close'], length=13)
    #df['ema_nine'] = ema(df['Close'], 9)
    #df['ema_thirteen'] = ema(df['Close'], 13)
    return df
    
def pull_intraday_data(ticker_symbol, today_date_str):
    intraday_data = yf.download(tickers=ticker_symbol, period='2d', interval='1m', auto_adjust=True, threads =False)
    #print(shared._ERRORS)
    sleep(0.06)
    intraday_data = resample_data(intraday_data, '3T')
    intraday_data = calculate_indicators(intraday_data)
    intraday_data = intraday_data.loc[today_date_str]
    
    return intraday_data

# Initialize intraday strategy columns
def initialize_columns(intraday_df):
    # Create columns for Triple EMA
    intraday_df['triple_ema_low_closeness'] = 0
    intraday_df['triple_ema_high_closeness'] = 0
    intraday_df['triple_ema_hold_rule_one'] = 0
    intraday_df['triple_ema_retest_rule_two'] = 0
    intraday_df['triple_ema_hold_above_signal'] = 0
    intraday_df['triple_ema_hold_above_signal_cummulative'] = 0
    intraday_df['triple_ema_hold_below_signal'] = 0
    intraday_df['triple_ema_hold_below_signal_cummulative'] = 0 

    # Create columns for EMA 9 and VWAP crossover
    intraday_df['crossover_signal'] = 0
    intraday_df['crossover_position'] = 0

    # Create columns for vwap_retest
    intraday_df['vwap_retest_signal_rule_one'] = 0
    intraday_df['vwap_confirmation_candle_signal_rule_two'] = 0
    intraday_df['candle_open_close_midpoint'] = 0
    intraday_df['current_vwap_local_max'] = 0
    intraday_df['prev_vwap_local_max'] = 0
    intraday_df['current_vwap_local_min'] = np.inf
    intraday_df['prev_vwap_local_min'] = np.inf
    
    intraday_df['soft_stop_loss'] = 0
    
    return intraday_df
    


In [4]:
def generate_ema9_vwap_signal(df):
    # Looking for EMA9 and VWAP Crossover
    #df_temp = df.copy()
    #df_temp['crossover_signal'] = 0
    print("Running Analysis - EMA9 + VWAP Crossover Setup")
    
    df['crossover_signal'] = np.where(df['ema_nine'] > df['vwap'], 1, 0)
    df['crossover_position'] = df['crossover_signal'].diff()    
    return df

def hold_above_or_below(df):
    
    close_to_ema_tolerance = 0.0005 # 0.05%
    
    df = df.copy()
    
    # Check if last 5 candles are within tolerance    
    df['triple_ema_low_closeness'] = abs(df['Low'] - df['ema_nine']) / df['ema_nine']
    df['triple_ema_hold_above_signal'] = np.where((df['Low'] >= df['ema_nine']) | (df['triple_ema_low_closeness'] <= close_to_ema_tolerance) , 1, 0)
    #df['triple_ema_hold_above_signal'] = np.where(df['Low'] >= df['ema_nine'], 1, 0)
    df['triple_ema_hold_above_signal_cummulative'] = df['triple_ema_hold_above_signal'].cumsum()
    if (df.loc[df.index[-1], 'triple_ema_hold_above_signal_cummulative'] == 5):
        df.loc[df.index[-1], 'triple_ema_hold_rule_one'] = 1
        
        
    # Check if rule 1 fulfilled - 5 Candles below EMA 20
    df['triple_ema_high_closeness'] = abs(df['High'] - df['ema_nine']) / df['ema_nine']
    df['triple_ema_hold_below_signal'] = np.where((df['High'] <= df['ema_nine']) | (df['triple_ema_high_closeness'] <= close_to_ema_tolerance), -1, 0)
    #df['triple_ema_hold_below_signal'] = np.where(df['High'] <= df['ema_nine'], -1, 0)

    df['triple_ema_hold_below_signal_cummulative'] = df['triple_ema_hold_below_signal'].cumsum()
    if (df.loc[df.index[-1], 'triple_ema_hold_below_signal_cummulative'] == -5):
        df.loc[df.index[-1], 'triple_ema_hold_rule_one'] = -1
    
    tolerance  = 0.0005 # 0.05%
    # Check for Long or Short
    if df.loc[df.index[-1], 'triple_ema_hold_rule_one'] == -1:
    
        # Check if rule 2 fulfilled - candle 0 touches/nears EMA20, 0.05% is considered near
        if 0 <= (abs(df.loc[df.index[-1], 'ema_nine'] - df.loc[df.index[-1], 'High'])
                 /df.loc[df.index[-1], 'ema_nine']) <= tolerance:
            df.loc[df.index[-1], 'triple_ema_retest_rule_two'] = -1 
                
    
    elif df.loc[df.index[-1], 'triple_ema_hold_rule_one'] == 1:
        
        # Check if rule 2 fulfilled - candle 0 touches/nears EMA20, 0.05% is considered near
        if 0 <= (abs(df.loc[df.index[-1], 'ema_nine'] - df.loc[df.index[-1], 'Low'])
                 /df.loc[df.index[-1], 'ema_nine']) <= tolerance:
            df.loc[df.index[-1], 'triple_ema_retest_rule_two'] = 1
    
    return df
    
def triple_ema(df):
    #df_temp = df.copy()
    #df_temp['hold_rule_one'] = 0
    #df_temp['retest_rule_two'] = 0
    
    if(df.shape[0] >= 5):
        print("Running Analysis - TripleEMA Setup")
        # Check if last 5 candles are above EMA9
        #df_last_5 = df.iloc[-5:]
        df_last_5 = hold_above_or_below(df.iloc[-5:])
        # Update last row in main df
        df.iloc[-1] = df_last_5.iloc[-1]

    return df
    
    
def vwap_retest(df):
    print("Running Analysis - VWAP Retest Setup")
    # Check for vwap retest
    tolerance  = 0.001 # 0.1%
    vwap_candle_threshold = 8
    candlewick_midpoint = (df.loc[df.index[-1], 'Open'] + df.loc[df.index[-1], 'Close'])/2
    df.loc[df.index[-1], 'candle_open_close_midpoint'] = candlewick_midpoint
    #candle_midpoint = abs(df.loc[df.index[-1], 'Close'] + df.loc[df.index[-1], 'Open'])/2
    
    # Update Local Max and Min value
    if df.loc[df.index[-1], 'High'] > df.loc[df.index[-1], 'current_vwap_local_max']:
        df.loc[df.index[-1], 'current_vwap_local_max'] = df.loc[df.index[-1], 'High']
        
    if df.loc[df.index[-1], 'Low'] < df.loc[df.index[-1], 'current_vwap_local_min']:
        df.loc[df.index[-1], 'current_vwap_local_min'] = df.loc[df.index[-1], 'Low']
        
    if df.shape[0] >= vwap_candle_threshold :
        # Determine the last 5 candles, whether they are above or below vwap
        df_last_5 = df.iloc[-vwap_candle_threshold:]
        no_last_5_candle_pos_above = df_last_5[df_last_5['candle_open_close_midpoint'] >= df_last_5['vwap']].shape[0]
        no_last_5_candle_pos_below = df_last_5[df_last_5['candle_open_close_midpoint'] <= df_last_5['vwap']].shape[0]
        
        # Determine the last two candles' price action
        price_action_direction  = df.loc[df.index[-1], 'Close'] - df.loc[df.index[-2], 'Close']
        
        # When candle is above vwap and price action is still decreasing, it's in the direction to retest vwap
        if no_last_5_candle_pos_above >= int(round(vwap_candle_threshold*0.6)) and  price_action_direction < 0:
            #print("Enter LONG VWAP Retest")
            # Check if the candle's close / low is near vwap
            if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Low'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance):
                    df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = 1
                    
        # When candle is below vwap and price action is still increasing, it's in the direction to retest vwap
        #candlewick_midpoint <= df.loc[df.index[-1], 'vwap']
        elif no_last_5_candle_pos_below >= int(round(vwap_candle_threshold*0.6)) and  price_action_direction > 0:
            # Check if candle's close / high is near vwap
            if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'High'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance):
                    df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = -1
        
        # Only if vwap retest rule 1 is fulfilled, check for confirmation
        if df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] != 0:
            # For Long setup, check if latest candle's close is above the previous candle's close and it closes above vwap
            if (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == 1) and (df.loc[df.index[-1], 'Close'] > df.loc[df.index[-2], 'Close']) and (df.loc[df.index[-1], 'Close'] > df.loc[df.index[-1], 'vwap']):
                    df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = 1
            # For Short setup, check if confilatestrmation candle's close is below the previous candle's close and it closes below vwap
            elif (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == -1) and (df.loc[df.index[-1], 'Close'] < df.loc[df.index[-2], 'Close']) and (df.loc[df.index[-1], 'Close'] < df.loc[df.index[-1], 'vwap']):
                    df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = -1
            
#         # Check if previous candle has fulfilled vwap_test
#         if df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == 0:
#             # Check if latest candle is above or below vwap (using midpoint between HIGH and LOW to decide)
#             # and the price action is on a downtrend
#             # Candle above vwap (Long)
#             if candlewick_midpoint > df.loc[df.index[-1], 'vwap'] :
#                 # Check if candle's low is near vwap
#                 #if (0 <= (abs(df.loc[df.index[-1], 'Low'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance)  or (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Open'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 and price_action_direction < 0:
#                     df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = 1

#             # Candle below vwap (Short) and price action is on a uptrend
#             elif candlewick_midpoint < df.loc[df.index[-1], 'vwap'] :
#                 # Check if candle's high is near vwap
#                 #if (0 <= (abs(df.loc[df.index[-1], 'High'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Open'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 and price_action_direction > 0:
#                     df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = -1

#         else:
#             # For Long setup, check if latest candle's close is above the previous candle's close
#             if (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == 1) and (df.loc[df.index[-1], 'Close'] > df.loc[df.index[-2], 'Close']):
#                     df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
#             # For Short setup, check if confilatestrmation candle's close is below the previous candle's close
#             elif (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == -1) and (df.loc[df.index[-1], 'Close'] < df.loc[df.index[-2], 'Close']):
#                     df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
    
    # Append to result dataframe
    #result_df = result_df.append(df.iloc[-1])
    return df    

In [5]:
def input_data_dict (time, ticker_name, position_type, strategy_type, additional_info):
    # Initialize empty dict
    headings =  ["TIME", "TICKER", "POSITION TYPE", "STRATEGY TYPE", "ADDITIONAL INFO"]
    data_dict = dict.fromkeys(headings)
    # Populate with results
    data_dict['TIME'] = time
    data_dict['TICKER'] = ticker_name
    data_dict['POSITION TYPE'] = position_type
    data_dict['STRATEGY TYPE'] = strategy_type
    data_dict['ADDITIONAL INFO'] = additional_info
    
    return data_dict

def dashboard_output_df(key, last_row_result_df, data):
    headings =  ["TIME", "TICKER", "POSITION TYPE", "STRATEGY TYPE", "ADDITIONAL INFO"]
    data_dict = dict.fromkeys(headings)
    
    # Check for trading signals
    # Triple EMA rule 1 LONG
    if last_row_result_df.triple_ema_hold_rule_one[0] == 1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "LONG", "Triple EMA", "Rule 1 Pass - Last 5 Candles above EMA9")
        data.append(data_dict)
    #  Triple EMA rule 1 SHORT
    elif last_row_result_df.triple_ema_hold_rule_one[0] == -1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "SHORT", "Triple EMA", "Rule 1 Pass - Last 5 Candles below EMA9")
        data.append(data_dict)
    # Triple EMA rule 2 LONG
    if last_row_result_df.triple_ema_retest_rule_two[0] == 1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "LONG", "Triple EMA", "Rule 2 Pass - Retest at EMA20")
        data.append(data_dict)
    #  Triple EMA rule 1 SHORT
    elif last_row_result_df.triple_ema_retest_rule_two[0] == -1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "SHORT", "Triple EMA", "Rule 2 Pass - Retest at EMA20")
        data.append(data_dict)
        
    # EMA9 + VWAP COMBO 
    if last_row_result_df.crossover_position[0] == 1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "LONG", "EMA9 + VWAP Crossover", "EMA9 Crosses Above")
        data.append(data_dict)
    elif last_row_result_df.triple_ema_retest_rule_two[0] == -1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "SHORT", "EMA9 + VWAP Crossover", "EMA9 Crosses Below")
        data.append(data_dict)
    
    # VWAP rule 1 Retest
    if last_row_result_df.vwap_retest_signal_rule_one[0] == 1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "LONG", "VWAP", "Rule 1 Pass - Retest at VWAP")
        data.append(data_dict)
    elif last_row_result_df.vwap_retest_signal_rule_one[0] == -1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "SHORT", "VWAP", "Rule 1 Pass - Retest at VWAP")
        data.append(data_dict)
        
    # VWAP rule 2 confirmation candle
    if last_row_result_df.vwap_confirmation_candle_signal_rule_two[0] == 1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "LONG", "VWAP", "Rule 2 Pass - Confirmation Candle")
        data.append(data_dict)
    elif last_row_result_df.vwap_confirmation_candle_signal_rule_two[0] == -1:
        data_dict = input_data_dict(last_row_result_df.index[0], key, "SHORT", "VWAP", "Rule 2 Pass - Confirmation Candle")
        data.append(data_dict)
        
    
    #print(last_row_result_df.index[0])
    #data_dict = input_data_dict(last_row_result_df.index[0], key, "LONG", "TRIPLE EMA", "Last 5 Candles above EMA9")
    #data.append(data_dict)
    
    return data
    
def run_strategy(n, data, watchlist_dict):
    today_date = datetime.date(2022, 4, 12) # to be replaced by datetime.now()
    today_date_str = today_date.strftime("%Y-%m-%d")
    
    interval = 3
    print(n)
    datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=n*interval) 
    # Iterate through watchlist tickers
    for key, df in watchlist_dict.items():
        print(datetime_time, "- Running Analysis on Ticker Symbol: ", key)
        # Pull Live Data
        intraday_df_today = pull_intraday_data(key, today_date_str)
#         #print(intraday_df_today.tail())
        if intraday_df_today.shape[0] > 0:
            intraday_df_today = initialize_columns(intraday_df_today)
            #print(intraday_df_today.head())        

            result_df = watchlist_dict[key]
            time_list = []
            triple_ema_time_list = []
            ema9_crossover_time_list = []
            vwap_time_list = []

            # Narrow to current index
            #datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=index*interval) 
            #eastern_tzinfo = pytz.timezone("US/Eastern")
            #datetime_time = eastern_tzinfo.localize(datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=index*interval), is_dst=None)
            # Convert datetime to string
            #datetime_time_str = datetime_time.strftime('%Y-%m-%d %H:%M:%S%z')
            #print(datetime_time)                                        
            intraday_df = intraday_df_today.loc[:datetime_time]

            # Append latest row to result_df for processing
            result_df = result_df.append(intraday_df.iloc[-1])
            result_df['vwap_confirmation_candle_signal_rule_two'] = result_df['vwap_confirmation_candle_signal_rule_two'].astype('bool')

            # Triple EMA
            temp_start_time = timeit.default_timer()
            result_df = triple_ema(result_df)
            #triple_ema_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())

            # EMA9 and VWAP crossover
            temp_start_time = timeit.default_timer()
            result_df = generate_ema9_vwap_signal(result_df)
            #ema9_crossover_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())

            # Test for VWAP retest
            temp_start_time = timeit.default_timer()
            result_df = vwap_retest(result_df)
            #vwap_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())
            #print(result_df.tail(5))
            #display(result_df.loc[result_df['vwap_retest_signal_rule_one'] == 1].head(10))
            #display(result_df.loc[result_df['vwap_confirmation_candle_signal_rule_two'] == True].head(10))

            #time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - start_time)).total_seconds())
            # Plot Graph
            #f = update_graph(intraday_df, f)
            dashboard_output_df(key, result_df.iloc[-1:], data)

            watchlist_dict[key] = result_df
            print("Successfully Ran Analysis on Ticker Symbol: ", key)
    return data

def live_run_strategy(n, data, watchlist_dict, live_counter):
    
    datetime_now = datetime.datetime.now(eastern_tzinfo)
    datetime_now_str = datetime_now.strftime('%Y-%m-%d %H:%M:%S')
    
    #today_date = datetime.date(2022, 3, 28) # to be replaced by datetime.now()
    today_date = datetime_now.date() # to be replaced by datetime.now()
    today_date_str = today_date.strftime("%Y-%m-%d")
   
    print(today_date_str)
    # Pull Live Spy Data
    spy_intraday_df_today = pull_intraday_data('SPY', today_date_str)
    spy_data_latest_datetime = spy_intraday_df_today.iloc[-1:].index[0]
    spy_data_latest_datetime_str = spy_data_latest_datetime.strftime('%Y-%m-%d %H:%M:%S')
    
    # Calculate Elapsed Time
    #spy_data_latest_datetime = datetime.datetime.combine(today_date, datetime.time(11, 57, 0)) 
    elapsed_time = spy_data_latest_datetime.tz_convert(tz=eastern_tzinfo) - eastern_tzinfo.localize(datetime.datetime.combine(today_date, datetime.time(9, 30, 0)), is_dst=None)

    # 3 Minutes
    interval = 3
    total_elapsed_intervals = elapsed_time.total_seconds()//(interval*60)
    
    if live_counter == None:
        live_counter = -1
#     elif live_counter < total_elapsed_intervals:
#         live_counter += 1
        
#     print(n, live_counter, total_elapsed_intervals)
#     datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=live_counter*interval)
    
    # Run Strategy if only live_counter needs catching up
    if live_counter < total_elapsed_intervals:
        live_counter += 1
        
        print(n, live_counter, total_elapsed_intervals)
        datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=live_counter*interval)
        
        print(datetime_now_str, "- Running ", datetime_time, " Analysis on Watchlist.")
        # Iterate through watchlist tickers
        for key, df in watchlist_dict.items():
            print(datetime_now_str, "- Running ", datetime_time, " Analysis on Ticker Symbol: ", key)
            # Pull Live Data
            intraday_df_today = pull_intraday_data(key, today_date_str)
            data_latest_datetime = intraday_df_today.iloc[-1:].index[0]
            data_latest_datetime_str = data_latest_datetime.strftime('%Y-%m-%d %H:%M:%S')

            if (intraday_df_today.shape[0] > 0) :
                intraday_df_today = initialize_columns(intraday_df_today)

                result_df = watchlist_dict[key]
                time_list = []
                triple_ema_time_list = []
                ema9_crossover_time_list = []
                vwap_time_list = []

                # Narrow down to datetime based on live_counter
                intraday_df = intraday_df_today.loc[:datetime_time]

                # Append latest row to result_df for processing
                result_df = result_df.append(intraday_df.iloc[-1])
                result_df['vwap_confirmation_candle_signal_rule_two'] = result_df['vwap_confirmation_candle_signal_rule_two'].astype('bool')

                # Triple EMA
                temp_start_time = timeit.default_timer()
                result_df = triple_ema(result_df)
                #triple_ema_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())

                # EMA9 and VWAP crossover
                temp_start_time = timeit.default_timer()
                result_df = generate_ema9_vwap_signal(result_df)
                #ema9_crossover_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())

                # Test for VWAP retest
                temp_start_time = timeit.default_timer()
                result_df = vwap_retest(result_df)
                #vwap_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())

                #time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - start_time)).total_seconds())
                # Plot Graph
                #f = update_graph(intraday_df, f)
                dashboard_output_df(key, result_df.iloc[-1:], data)

                watchlist_dict[key] = result_df
    else:
        print(datetime_now_str, "- Waiting for the next ", interval, " minutes interval...")
    return data, live_counter
    

In [6]:
import dash
from dash import dcc, html
import plotly
from dash import Dash, html, Input, Output, State, dash_table
import pandas as pd
import plotly.graph_objects as go

# Read and Create Watchlist Dictionary
watchlist_dict = create_watchlist_dict("./Watchlist_test.csv")
eastern_tzinfo = pytz.timezone("US/Eastern")

# Initialize Dashboard Data Dataframe
headings =  ["TIME", "TICKER", "POSITION TYPE", "STRATEGY TYPE", "ADDITIONAL INFO"]
df_data = pd.DataFrame(columns=headings)

#df_data.loc[len(df_data)] = ["24/02/2022 9:33PM", "MSFT", "LONG", "Triple EMA", "5 candles above"]
#df_data.loc[len(df_data)] = ["24/02/2022 9:33PM", "TSLA", "SHORT", "EMA9 VWAP Crossover", None]
#print(df_data.to_dict('records'))    
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div(id='live-time-update-text',
             style={'font-size': '50px', 'textAlign': 'center'}),
    html.H4('Stocks Scanner Live Feed Table - 3 Minutes Interval'),
    #html.P(id='data_table'),
    dash_table.DataTable(
        id='data_table',
        columns=[{"name": i, "id": i} 
                 for i in headings],
        data= df_data.to_dict('records'),
        #data=df.to_dict('records'),
        style_cell=dict(textAlign='left'),
        style_header={
            'backgroundColor': '#191970',
            'color': 'white'},
        style_data=dict(backgroundColor="#FF8C00"),
        fixed_rows={'headers': True},
        style_table={'height': '300px', 'overflowY': 'auto'},
        style_data_conditional=[]
    ),
    dcc.Interval(
        id='interval-component',
        interval=5*1000, # in milliseconds
        n_intervals=0
    ),
    dcc.Interval(
        id='live-time-interval-component',
        interval=1000 # in milliseconds
    ),
    dcc.Store(id='store_local', storage_type='session'),
    dcc.Graph(id='live-candlestick-graph'),
    html.Div(id='output_div')
])

# Get Active Cell Click
@app.callback(
    [Output('output_div', 'children'),
    Output('live-candlestick-graph', 'figure')],
    Input('data_table', 'active_cell'),
    State('data_table', 'data')
)
def getActiveCell(active_cell, data):
    if active_cell:
        col = active_cell['column_id']
        row = active_cell['row']
        cellData = data[row][col]
        timestmap = data[row]['TIME']
        ticker = data[row]['TICKER']
        
        # Extract Dataframe with watchlist dictionary
        result_df = watchlist_dict[ticker]
        
        # Narrow to active cell's timestamp
        _narrow_result_df = result_df.loc[:timestmap]
        
        fig = go.Figure(go.Candlestick(
            x=_narrow_result_df.index,
            open=_narrow_result_df['Open'],
            high=_narrow_result_df['High'],
            low=_narrow_result_df['Low'],
            close=_narrow_result_df['Close'],
            name = ticker
        ))
        
        fig.add_trace(go.Scatter(x=_narrow_result_df.index, y=_narrow_result_df['vwap'], mode='lines', name = 'VWAP', 
                  line=dict(color='teal')))
        fig.add_trace(go.Scatter(x=_narrow_result_df.index, y=_narrow_result_df['ema_nine'], mode='lines', name = 'EMA9', 
                  line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=_narrow_result_df.index, y=_narrow_result_df['ema_thirteen'], mode='lines', name = 'EMA13', 
                  line=dict(color='purple')))
        
        fig.update_xaxes(
            rangeslider_visible=True,
            rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2020-12-25", "2021-01-01"])  # hide holidays (Christmas and New Year's, etc)
            ]
        )
        
        return html.P(f'row: {row}, col: {col}, value: {cellData}, timestamp: {timestmap}, ticker: {ticker}'), fig
    return html.P('no cell selected')

# Live Time Update
@app.callback(Output('live-time-update-text', 'children'),
              [Input('live-time-interval-component', 'n_intervals')])
def update_date(n_int):
      return [html.P(datetime.datetime.now(eastern_tzinfo).strftime('%Y-%m-%d %H:%M:%S'))]

# Run Strategy    
@app.callback(
    [Output('data_table', 'data'),
    Output('data_table', 'style_data_conditional'),
    Output('store_local', 'data')],
    Input('interval-component', 'n_intervals'),
    [State('data_table', 'data'),
    State('store_local', 'data')]
)
def updateData(n, data, live_counter):
    #if n > -1:
    # Run Strategy
    #data = run_strategy(n, data, watchlist_dict)
    data, live_counter = live_run_strategy(n, data, watchlist_dict, live_counter)
    #data.append({'TIME': "24/02/2022 9:36PM", 'TICKER': "TSLA", 'POSITION TYPE': "SHORT", 'STRATEGY TYPE':"EMA9 VWAP Crossover", 'ADDITIONAL INFO':None})
    #data.append({'TIME': "24/02/2022 9:36PM", 'TICKER': "MSFT", 'POSITION TYPE': "LONG", 'STRATEGY TYPE':"EMA9 VWAP Crossover", 'ADDITIONAL INFO':None})
    df_data = pd.DataFrame.from_records(data)
#     df_data['TIME'] = pd.to_datetime(
#                           df_data['TIME'],
#                           format='%Y-%m-%d %H:%M:%S%z'
#     )
    default_cond_style = []
    if(df_data.shape[0]>0):
        # Update the latest datetime entries with different colour
        data_latest_datetime = df_data['TIME'].iloc[-1]
        data_latest_datetime_cast = pd.Timestamp(data_latest_datetime)
        #print(data_latest_datetime_cast, type(data_latest_datetime_cast))
        # Cast to String
        date_time_now_str = data_latest_datetime_cast.strftime('%Y-%m-%dT%H:%M:%S%z')
        #print(data_latest_datetime_cast, date_time_now_str)
        # Refer to https://www.rapidtables.com/web/css/css-color.html#blue for color
        default_cond_style = [
            {'if': {'filter_query': '{{TIME}} = {}'.format(date_time_now_str)},
            'backgroundColor': '#FF8C00',
                'color': 'black'},
            {'if': {'filter_query': '{{TIME}} < {}'.format(date_time_now_str),},
            'backgroundColor': "#778899",
                'color': 'white'}
        ]
        
    return data, default_cond_style, live_counter

if __name__ == '__main__':        
    app.run_server(debug=False, use_reloader=False)    
    

  Exchange Symbol
0   NASDAQ   MSFT
1   NASDAQ   AAPL
2   NASDAQ   TSLA
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\rsct\AppData\Local\Temp/ipykernel_23028/3942779979.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  watchlist_dict = pd.Series(None,df_watchlist.Symbol.values).to_dict()
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2022 22:37:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:37:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:37:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:37:24] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2022 22:37:24] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2022 22:37:24] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2022 22:37:24] "GET /_dash-component-suites/dash/dcc/async-plotlyjs

2022-04-13
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\rsct\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\rsct\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\rsct\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\rsct\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\rsct\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\rsct\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\rsct\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.s

127.0.0.1 - - [13/Apr/2022 22:37:24] "POST /_dash-update-component HTTP/1.1" 500 -


[*********************100%***********************]  1 of 1 completed
0 0 22.0
2022-04-13 10:37:24 - Running  2022-04-13 09:30:00  Analysis on Watchlist.
2022-04-13 10:37:24 - Running  2022-04-13 09:30:00  Analysis on Ticker Symbol:  MSFT
[*********************100%***********************]  1 of 1 completed


...

127.0.0.1 - - [13/Apr/2022 22:39:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:39:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:39:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:39:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:39:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:39:44] "POST /_dash-update-component HTTP/1.1" 200 -


2022-04-13
[*********************100%***********************]  1 of 1 completed


127.0.0.1 - - [13/Apr/2022 22:39:45] "POST /_dash-update-component HTTP/1.1" 200 -


2022-04-13 10:39:44 - Waiting for the next  3  minutes interval...


127.0.0.1 - - [13/Apr/2022 22:39:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:39:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2022 22:39:47] "POST /_dash-update-component HTTP/1.1" 200 -


In [8]:
from statistics import mean
from matplotlib import pyplot  as plt
%matplotlib

mean_time_taken = round(mean(time_list),4)

fig = plt.figure()
plt.plot(time_list,'g', label='All 3 Methods')
plt.plot(triple_ema_time_list,'b', label='Triple EMA')
plt.plot(ema9_crossover_time_list,'k', label='EMA9 + VWAP Combo')
plt.plot(vwap_time_list,'r', label='VWAP Retest')
plt.axhline(y = mean_time_taken, color = 'm', linestyle = '--')
plt.xlabel('Iterations')
plt.ylabel('Time Taken (seconds)')
plt.title('Processing time taken for 1 ticker across each iteration')
plt.grid(True)
plt.legend(["All 3 Methods", "Triple EMA", "EMA9 + VWAP Combo", "VWAP Retest", 'Mean Time Taken'], loc="upper left")
plt.show()

from statistics import mean
round(mean(time_list),4)

Using matplotlib backend: Qt5Agg


NameError: name 'time_list' is not defined

In [12]:
# Backtesting
import backtrader as bt
from strategies import *
import backtrader.feeds as btfeeds
#import quantstats
import math

today_date = datetime.date(2022, 4, 12) 
today_date_str = today_date.strftime("%Y-%m-%d")

spy_intraday_df_today = pull_intraday_data('TSLA', today_date_str) # 'ETH-USD', 'BTC-USD', 'SOL-USD'
spy_intraday_df_today = initialize_columns(spy_intraday_df_today)
#print(spy_intraday_df_today.head())
#print(spy_intraday_df_today.columns.values.tolist())

class CustomData(btfeeds.PandasData):
    lines = ('ema_thirteen', 'vwap', 'candle_open_close_midpoint', 'current_vwap_local_max', 'current_vwap_local_min',
            'vwap_retest_signal_rule_one', 'vwap_confirmation_candle_signal_rule_two', 'soft_stop_loss')
    params = (
        ('datetime', None),
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', 'Volume'),
        ('openinterest', None),
        ('ema_thirteen', 'ema_thirteen'),
        ('vwap', 'vwap'),
        ('candle_open_close_midpoint', 'candle_open_close_midpoint'),
        ('current_vwap_local_max', 'current_vwap_local_max'),
        ('current_vwap_local_min', 'current_vwap_local_min'),
        ('vwap_retest_signal_rule_one', 'vwap_retest_signal_rule_one'),
        ('vwap_confirmation_candle_signal_rule_two', 'vwap_confirmation_candle_signal_rule_two'),
        ('soft_stop_loss', 'soft_stop_loss')
        #('mark_price', 'mark_price')
    )
        
cerebro = bt.Cerebro()

#data = bt.feeds.PandasData(dataname=spy_intraday_df_today, tz=pytz.timezone('US/Eastern'))
data = CustomData(dataname=spy_intraday_df_today, tz=pytz.timezone('US/Eastern'))
cerebro.adddata(data)
cerebro.broker.setcash(10000.0)
cerebro.broker.set_shortcash(False)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Minutes, compression=3, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
#cerebro.addstrategy(EMAcrossover)
cerebro.addstrategy(VWAPretest)

# Run Cerebro Engine
start_portfolio_value = cerebro.broker.getvalue()
run = cerebro.run()[0]

end_portfolio_value = cerebro.broker.getvalue()
sharpe = run.analyzers.sharpe_ratio.get_analysis()
returns = run.analyzers.returns.get_analysis()
pnl = end_portfolio_value - start_portfolio_value
print(f'Starting Portfolio Value: {start_portfolio_value:2f}')
print(f'Final Portfolio Value: {end_portfolio_value:2f}')
print(f'Total Returns: {pnl:.2f}')
print('Sharpe Ratio (Snapshot per day): ', round(sharpe['sharperatio'], 2))
print('Returns: ', round(returns['rtot'], 2))

[*********************100%***********************]  1 of 1 completed
----------------------------------------------------------------------------
2022-04-12 11:33:00 SELL CREATED at price: 1000.549988, stop loss: 1007.187726, position size: 8
2022-04-12 11:36:00 SELL EXECUTED, Price: 1000.70, Cost: 8005.56, Comm: 0.00
2022-04-12 11:51:00 CROSSES ABOVE EMA13, CLOSE CREATE 1001.390015
2022-04-12 11:54:00 BUY EXECUTED, Price: 1001.49, Cost: 8005.56, Comm: 0.00
2022-04-12 11:54:00 OPERATION PROFIT, GROSS -6.36, NET -6.36
----------------------------------------------------------------------------
2022-04-12 13:15:00 SELL CREATED at price: 999.500000, stop loss: 1005.741633, position size: 8
2022-04-12 13:18:00 SELL EXECUTED, Price: 999.43, Cost: 7995.44, Comm: 0.00
2022-04-12 14:03:00 CROSSES ABOVE EMA13, CLOSE CREATE 993.633484
2022-04-12 14:06:00 BUY EXECUTED, Price: 993.46, Cost: 7995.44, Comm: 0.00
2022-04-12 14:06:00 OPERATION PROFIT, GROSS 47.76, NET 47.76
Starting Portfolio Value: 1